# Task 1.1 - Construct RLTK Datasets

In [40]:
# %pip install rltk

import rltk
import csv
import pandas as pd
import numpy as np

# Using pandas to inspect the dataframe and understand the data

source_2 = pd.read_csv('data/Sample - Nutrition Data Self - Source 2.csv')
source_1 = pd.read_csv('data/Sample - Nutrition Value - Source 1.csv')

source_2 = source_2.apply(pd.to_numeric, errors="ignore")
source_1 = source_1.apply(pd.to_numeric, errors="ignore")

print(source_2.columns)
print(source_1.columns)

print(source_2.describe())
print(source_1.describe())

source_2['Food Name'].replace('', np.nan, inplace=True)
source_1['Food Name'].replace('', np.nan, inplace=True)

source_2.dropna(subset=['Food Name'], inplace=True)
source_1.dropna(subset=['Food Name'], inplace=True)

Index(['Food Name', 'Glycemic Load', 'Completeness Score', 'Amino Acid Score'], dtype='object')
Index(['Ash', 'Calcium', 'Calories', 'Carbohydrate', 'Copper', 'Fat', 'Iron',
       'Magnesium', 'Potassium', 'Protein', 'Saturated fatty acids', 'Sodium',
       'Vitamin A, RAE', 'Vitamin B6', 'Water', 'Zinc', 'Food Name',
       'Fatty acids, total trans', 'Manganese', 'Vitamin D'],
      dtype='object')
       Glycemic Load  Completeness Score  Amino Acid Score
count     535.000000          535.000000        340.000000
mean        8.082243           40.177570         93.402941
std        22.397682           24.313578         43.987857
min         0.000000            0.000000          0.000000
25%         0.000000           23.000000         68.000000
50%         2.000000           37.000000         94.500000
75%         8.000000           55.000000        133.250000
max       304.000000           93.000000        181.000000
              Ash      Calcium     Calories  Carbohydrate      

In [43]:
# Create record schema for both data sources

tokenizer = rltk.tokenizer.crf_tokenizer.crf_tokenizer.CrfTokenizer()
import re

class Source1Record(rltk.Record):
    def __init__(self, raw_object):
        super().__init__(raw_object)
        self.name = ''
    
    @rltk.cached_property
    def id(self):
        return self.raw_object['Food Name']

    @rltk.cached_property
    def name_string(self):
        if(len(self.raw_object['Food Name']) == 0):
            return ' '
        else:
            return str(self.raw_object['Food Name'])
    
    @rltk.cached_property
    def name_tokens(self):
        return set(tokenizer.tokenize(self.name_string))
    
    @rltk.cached_property
    def Ash(self):
        return self.raw_object['Ash']
    
    @rltk.cached_property
    def Calcium(self):
        return self.raw_object['Calcium']
    
    @rltk.cached_property
    def Calories(self):
        return self.raw_object['Calories']
    
    @rltk.cached_property
    def Carbohydrate(self):
        return self.raw_object['Carbohydrate']
    
    @rltk.cached_property
    def Copper(self):
        return self.raw_object['Copper']
    
    @rltk.cached_property
    def Fat(self):
        return self.raw_object['Fat']
    
    @rltk.cached_property
    def Iron(self):
        return self.raw_object['Iron']
    
    @rltk.cached_property
    def Magnesium(self):
        return self.raw_object['Magnesium']
    
    @rltk.cached_property
    def Potassium(self):
        return self.raw_object['Potassium']
    
    @rltk.cached_property
    def Protein(self):
        return self.raw_object['Protein']

    @rltk.cached_property
    def Saturated_fatty_acids(self):
        return self.raw_object['Saturated fatty acids']
    
    @rltk.cached_property
    def Sodium(self):
        return self.raw_object['Sodium']
    
    @rltk.cached_property
    def Vitamin_A_RAE(self):
        return self.raw_object['Vitamin A, RAE']
    
    @rltk.cached_property
    def Vitamin_B6(self):
        return self.raw_object['Vitamin B6']
    
    @rltk.cached_property
    def Water(self):
        return self.raw_object['Water']
    
    @rltk.cached_property
    def Zinc(self):
        return self.raw_object['Zinc']
    
    @rltk.cached_property
    def Fatty_acids_total_trans(self):
        return self.raw_object['Fatty acids, total trans']
    
    @rltk.cached_property
    def Manganese(self):
        return self.raw_object['Manganese']
    
    @rltk.cached_property
    def Vitamin_D(self):
        return self.raw_object['Vitamin D']

class Source2Record(rltk.Record):
    def __init__(self, raw_object):
        super().__init__(raw_object)
        self.name = ''
    
    @rltk.cached_property
    def id(self):
        return self.raw_object['Food Name']

    @rltk.cached_property
    def name_string(self):
        if(len(self.raw_object['Food Name']) == 0):
            return ' '
        else:
            return str(self.raw_object['Food Name'])
    
    @rltk.cached_property
    def name_tokens(self):
        return set(tokenizer.tokenize(self.name_string))

    @rltk.cached_property
    def Glycemic_Load(self):
        return self.raw_object['Glycemic Load']

    @rltk.cached_property
    def Completeness_Score(self):
        return self.raw_object['Completeness Score']

    @rltk.cached_property
    def Amino_Acid_Score(self):
        return self.raw_object['Amino Acid Score']

In [44]:
ds1 = rltk.Dataset(rltk.CSVReader('data/Sample - Nutrition Value - Source 1.csv'), record_class=Source1Record)
ds2 = rltk.Dataset(rltk.CSVReader('data/Sample - Nutrition Data Self - Source 2.csv'), record_class=Source2Record)

# Task 1.2 - Blocking

In [49]:
# Cleaning pipeline and comparision functions

import string
import math

def toLower(text):
    return str(text).lower()

def extraSpaces(text):
    return re.sub(' +', ' ', text, flags=re.MULTILINE)

def removeNum(text):
    return re.sub(r'[0-9]+', '', text)

def removePunct(text):
    return text.translate(str.maketrans('', '', string.punctuation))

def basicCleaningPipeline(s):
    s1 = toLower(s)
    s1 = removePunct(s1)
    s1 = removeNum(s1)
    s1 = extraSpaces(s1)
    return s1

def jaccard(r1, r2):
    s1 = r1.name_tokens
    s2 = r1.name_tokens
    
    return rltk.jaccard_index_similarity(s1, s2)

def page(r1, r2):
    p1 = r1.pages
    p2 = r1.pages

    if(abs(p1-p2) < 3):
        return 1
    else:
        return 0

In [51]:
def compound(name):
    name = basicCleaningPipeline(name)
    size = len(name.split())
    label = str(size-1) + '-' + str(size+1)
    print(label)
    return label

bg = rltk.HashBlockGenerator()
block = bg.generate(
    bg.block(ds1, function_=lambda x: compound(x.name_string)),
    bg.block(ds2, function_=lambda x: compound(x.name_string))
)

2-4
5-7
4-6
6-8
7-9
4-6
10-12
6-8
5-7
4-6
1-3
4-6
12-14
10-12
5-7
10-12
8-10
8-10
12-14
10-12
9-11
7-9
6-8
8-10
12-14
4-6
3-5
2-4
4-6
5-7
5-7
3-5
9-11
5-7
8-10
4-6
9-11
9-11
1-3
8-10
6-8
13-15
3-5
5-7
5-7
5-7
4-6
3-5
8-10
4-6
5-7
5-7
4-6
12-14
7-9
2-4
4-6
8-10
4-6
5-7
6-8
5-7
10-12
8-10
4-6
4-6
8-10
5-7
7-9
4-6
7-9
3-5
3-5
5-7
13-15
8-10
8-10
11-13
1-3
5-7
3-5
10-12
5-7
12-14
6-8
8-10
5-7
12-14
11-13
5-7
7-9
10-12
9-11
9-11
4-6
11-13
10-12
6-8
2-4
10-12
10-12
7-9
1-3
6-8
11-13
1-3
7-9
5-7
5-7
4-6
8-10
12-14
3-5
11-13
11-13
12-14
2-4
7-9
5-7
5-7
13-15
11-13
12-14
11-13
4-6
3-5
6-8
8-10
5-7
7-9
7-9
2-4
11-13
12-14
3-5
3-5
4-6
8-10
3-5
9-11
3-5
5-7
5-7
1-3
4-6
6-8
10-12
7-9
4-6
13-15
3-5
5-7
6-8
3-5
13-15
3-5
3-5
2-4
5-7
6-8
5-7
4-6
6-8
11-13
9-11
12-14
6-8
6-8
5-7
7-9
15-17
4-6
7-9
2-4
4-6
6-8
5-7
4-6
6-8
3-5
12-14
11-13
5-7
4-6
10-12
6-8
10-12
8-10
6-8
7-9
3-5
4-6
1-3
3-5
14-16
7-9
11-13
5-7
4-6
4-6
9-11
7-9
11-13
6-8
8-10
9-11
11-13
1-3
7-9
10-12
13-15
7-9
2-4
8-10
2-4
10-12
3-5
8-10
1

# Task 1.3 - Entity Linking

In [52]:

def perfect_match(r1, r2):
    name1 = basicCleaningPipeline(r1.name_string)
    name2 = basicCleaningPipeline(r2.name_string)

    if(name1 == name2):
        return 1
    
    fuzzy_thresh = 0.85

    name_l_score = round(rltk.levenshtein_similarity(name1, name2), 2)

    if(name_l_score > fuzzy_thresh):
        return 1
    
    return 0

In [53]:
# Rule based method

MY_TRESH = 0.85 

def rule_based_method(r1, r2):
    jaccard_score = jaccard(r1, r2)
    perfect_match_score = perfect_match(r1, r2)
    
    total = 0.20 * jaccard_score + \
            0.80 * perfect_match_score
    
    # return two values: boolean if they match or not, float to determine confidence
    return round(total, 2) > MY_TRESH, total

# Task 1.4 - Record Linkage

In [60]:
"""
Header:
Name,Ash,Calcium,Calories,Carbohydrate,Copper,Fat,Iron,Magnesium,Potassium,Protein,Saturated_fatty_acids,Sodium,Vitamin_A_RAE,VItamin_B6,Water,Zinc,Fatty_acids_total_trans,Manganese,Vitamin_D,Glycemic_Load,Completeness_Score,Amino_Acid_Score

From 18 mins to 2 mins for 10Kx10K records with blocking. That is ~10 times faster.
"""

true_predictions = list()

with open('merged_dataset.csv', mode='w') as file:
    writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    candidate_pairs = rltk.get_record_pairs(ds1, ds2, block=block)

    hit_count = 0

    for r1, r2 in candidate_pairs:
        result, confidence = rule_based_method(r1, r2)
        if(result):
            hit_count += 1
            true_predictions.append((r1.id, r2.id))
            print(r1.name_string, "===", r2.name_string, confidence)
            writer.writerow((r2.id, r1.Ash, r1.Calcium, r1.Calories, r1.Carbohydrate, r1.Copper, r1.Fat, r1.Iron, r1.Magnesium, r1.Potassium, r1.Protein, r1.Saturated_fatty_acids, r1.Sodium, r1.Vitamin_A_RAE, r1.Vitamin_B6, r1.Water, r1.Zinc, r1.Fatty_acids_total_trans, r1.Manganese, r1.Vitamin_D, r2.Glycemic_Load, r2.Completeness_Score, r2.Amino_Acid_Score))

print("Hits:", hit_count)

Seaweed, raw, kelp === seaweed raw kelp 0.8800000000000001
Broccoli, cooked, chinese === broccoli cooked chinese 0.8800000000000001
Soybeans, raw, green === soybeans raw green 0.8800000000000001
Chicken, boiled, feet === chicken boiled feet 0.8800000000000001
Fat, beef tallow === fat beef tallow 0.8800000000000001
Malabar spinach, cooked === malabar spinach cooked 0.8800000000000001
Seaweed, raw, agar === seaweed raw agar 0.8800000000000001
Mushrooms, grilled, portabella === mushrooms grilled portabella 0.8800000000000001
Olive loaf, pork === olive loaf pork 0.8800000000000001
Babyfood, pear, juice === babyfood pear juice 0.8800000000000001
Fiddlehead ferns, raw === fiddlehead ferns raw 0.8800000000000001
Chicory, raw, witloof === chicory raw witloof 0.8800000000000001
Ostrich, raw, tenderloin === ostrich raw tenderloin 0.8800000000000001
Onions, raw, welsh === onions raw welsh 0.8800000000000001
Seaweed, dried, spirulina === seaweed dried spirulina 0.8800000000000001
Bologna, pork, ch

'\nHeader:\nName,Ash,Calcium,Calories,Carbohydrate,Copper,Fat,Iron,Magnesium,Potassium,Protein,Saturated_fatty_acids,Sodium,Vitamin_A_RAE,VItamin_B6,Water,Zinc,Fatty_acids_total_trans,Manganese,Vitamin_D,Glycemic_Load,Completeness_Score,Amino_Acid_Score\n\nFrom 18 mins to 2 mins for 10Kx10K records with blocking. That is ~10 times faster.\n'

# Task 2

# Task 2.1 - Construct KG

In [61]:
# ! pip install rdflib

In [62]:
# Define namespaces

from rdflib import Graph, URIRef, Literal, XSD, Namespace, RDF

FOOD = Namespace('https://dbpedia.org/ontology/Food')
NUTRITIONINFORMATION = Namespace('https://schema.org/NutritionInformation')

In [63]:
# Bind to knowledge graph

my_kg = Graph()
my_kg.bind('food', FOOD)
my_kg.bind('nutriinfo', NUTRITIONINFORMATION)


In [64]:
# URI parser to replace space with hyfhen

def parseURI(identifier):
    return identifier.replace(' ', '-')

In [144]:
# Create nodes and insert into the KG

for index, pred in enumerate(true_predictions):
    source1_name = int(pred[0])
    source2_name = int(pred[1])

    g_title = source_2['Food Name'][source2_name]
    
    # book_node_uri = URIRef(BOOK[parseURI(g_title)])

    # my_kg.add((book_node_uri, RDF.type, BOOK['name']))
    # my_kg.add((book_node_uri, BOOK['name'], Literal(g_title, datatype=XSD.string)))

    # g_description = good_df['Description'][good_index] if len(str(good_df['Description'][good_index]).strip()) > 0 else 'N/A'
    # g_isbn13 = good_df['ISBN13'][good_index] if len(str(good_df['ISBN13'][good_index]).strip()) > 0 else barnes_df['ISBN13'][barnes_index]
    # g_pages = good_df['PageCount'][good_index] if len(str(good_df['PageCount'][good_index]).strip()) > 0 else barnes_df['Pages'][barnes_index]
    # g_author = good_df['FirstAuthor'][good_index] if len(str(good_df['FirstAuthor'][good_index]).strip()) > 0 else barnes_df['Author1'][barnes_index]

    # if(len(str(g_description).strip()) > 0):
    #     my_kg.add((book_node_uri, BOOK['abstract'], Literal(g_description, datatype=XSD.string)))
    # if(len(str(g_isbn13).strip()) > 0):
    #     my_kg.add((book_node_uri, BOOK['isbn'], Literal(g_isbn13, datatype=XSD.string)))
    # if(len(str(g_pages).strip()) > 0):
    #     my_kg.add((book_node_uri, BOOK['numberOfPages'], Literal(g_pages, datatype=XSD.positiveInteger)))

    # author_node_uri = URIRef(PERSON[parseURI(g_author)])

    # my_kg.add((author_node_uri, RDF.type, PERSON['name']))
    # my_kg.add((author_node_uri, PERSON['name'], Literal(g_author, datatype=XSD.string)))
    # my_kg.add((book_node_uri, BOOK['author'], author_node_uri))

    # g_rating_good = good_df['Rating'][good_index]
    # g_number_rating_good = good_df['NumberofRatings'][good_index]
    # g_number_review_good = good_df['NumberofReviews'][good_index]

    # g_sales_barnes = barnes_df['Salesrank'][barnes_index]
    # g_rating_count_barnes = barnes_df['Ratingscount'][barnes_index]
    # g_rating_value_barnes = barnes_df['Ratingvalue'][barnes_index]

    # if(len(str(g_rating_good).strip()) > 0 and len(str(g_rating_value_barnes).strip()) > 0):
    #     rating_node_uri = URIRef(RATING[parseURI(g_title)])
    #     my_kg.add((book_node_uri, RATING['aggregate'], Literal(str(g_rating_good)+"+"+str(g_rating_value_barnes))))
    #     my_kg.add((book_node_uri, RATING['good'], Literal(g_rating_good)))
    #     my_kg.add((book_node_uri, RATING['barnes'], Literal(g_rating_value_barnes)))

    #     my_kg.add((book_node_uri, RATING['goodNumRating'], Literal(g_number_rating_good)))
    #     my_kg.add((book_node_uri, RATING['goodNumReview'], Literal(g_number_review_good)))
    #     my_kg.add((book_node_uri, RATING['barnesNumRating'], Literal(g_rating_count_barnes)))
    #     my_kg.add((book_node_uri, RATING['barnesSale'], Literal(g_sales_barnes)))

    #     my_kg.add((book_node_uri, BOOK['aggregateRating'], rating_node_uri))


    # g_publisher = good_df['Publisher'][good_index] if len(str(good_df['Publisher'][good_index])) > 0 else barnes_df['Publisher'][barnes_index]
    # g_publish_date = good_df['PublishDate'][good_index] if len(str(good_df['PublishDate'][good_index])) > 0 else barnes_df['PublicationDate'][barnes_index]

    # if(len(str(g_publish_date).strip()) > 0):
    #     my_kg.add((book_node_uri, BOOK['datePublished'], Literal(g_publish_date, datatype=XSD.string)))

    # if(len(str(g_publisher)) > 0):
    #     publisher_node_uri = URIRef(PUBLISHER[parseURI(g_publisher)])
    #     my_kg.add((book_node_uri, BOOK['publisher'], rating_node_uri))
        

    # g_format = good_df['Format'][good_index]
    # g_lang = good_df['Language'][good_index]

    # g_paperback_price = barnes_df['Paperbackprice'][barnes_index].replace('$', '')
    # g_hardcover_price = barnes_df['Hardcoverprice'][barnes_index].replace('$', '')
    # g_nookbook_price = barnes_df['Nookbookprice'][barnes_index].replace('$', '')
    # g_audiobook_price = barnes_df['Audiobookprice'][barnes_index].replace('$', '')


    # price_node_uri = URIRef(PRICE[parseURI(g_title)])
    
    # if(len(str(g_paperback_price).strip()) > 0):
    #     my_kg.add((price_node_uri, PRICE['paperback'], Literal(float(g_paperback_price))))
    # if(len(str(g_hardcover_price).strip()) > 0):
    #     my_kg.add((price_node_uri, PRICE['hardcover'], Literal(float(g_hardcover_price))))
    # if(len(str(g_nookbook_price).strip()) > 0):
    #     my_kg.add((price_node_uri, PRICE['nookbook'], Literal(float(g_nookbook_price))))
    # if(len(str(g_audiobook_price).strip()) > 0):
    #     my_kg.add((price_node_uri, PRICE['audiobook'], Literal(float(g_audiobook_price))))

    # my_kg.add((book_node_uri, BOOK['cost'], rating_node_uri))


In [145]:
# Serialize graph and dump to file

my_kg.serialize('small_graph.ttl', format="turtle")

<Graph identifier=Na44baed45398445d82682a4d4d4e05af (<class 'rdflib.graph.Graph'>)>